In [3]:
import datasets

#load the japanese dataset
dataset_ja = datasets.load_dataset("cc100", lang="ja")

Using custom data configuration ja-lang=ja
Reusing dataset cc100 (/root/.cache/huggingface/datasets/cc100/ja-lang=ja/0.0.0/b583dd47b0dd43a3c3773075abd993be12d0eee93dbd2cfe15a0e4e94d481e80)


In [4]:
dataset_ja["train"][961563]

{'id': '961563', 'text': '彼女がホ○ルを凝視しているのを気が付かないふりをするなど誤魔化していました。\n'}

In [ ]:
from src.make_datasets import make_sentence_files

make_sentence_files(dataset_ja["train"])

writing to  /work/data/jp_sentences/sent_0.txt


100%|████████████████████████████████████████████████████████████████████████████████████████████| 10000000/10000000 [02:14<00:00, 74099.67it/s]


writing to  /work/data/jp_sentences/sent_1.txt


100%|████████████████████████████████████████████████████████████████████████████████████████████| 10000000/10000000 [02:12<00:00, 75483.89it/s]


writing to  /work/data/jp_sentences/sent_2.txt


100%|████████████████████████████████████████████████████████████████████████████████████████████| 10000000/10000000 [02:13<00:00, 74980.98it/s]


writing to  /work/data/jp_sentences/sent_3.txt


100%|████████████████████████████████████████████████████████████████████████████████████████████| 10000000/10000000 [02:10<00:00, 76620.94it/s]


writing to  /work/data/jp_sentences/sent_4.txt


100%|████████████████████████████████████████████████████████████████████████████████████████████| 10000000/10000000 [02:09<00:00, 77303.73it/s]

In [ ]:
# read text files as a list

import glob
import cfg

sentence_files = glob.glob(cfg.JP_SENTENCES_DIR + "/*.txt")
print(glob.glob(cfg.JP_SENTENCES_DIR + "/*.txt"))

In [ ]:
#load the english dataset
dataset_ja = datasets.load_dataset("cc100", lang="en")

In [ ]:
# changing the ja into jp here :)
import cfg
import sentencepiece as spm
import time
from multiprocessing import Pool


languages = ["jp", "en"]
sentences_fp = {"jp": cfg.JP_SENTENCES_FILE, "en": cfg.EN_SENTENCES_FILE}
vocab_sizes = [2000, 4000, 8000, 16000, 24000, 32000, 48000]
character_coverages = {"jp": 0.995, "en": 1.00}
multiproc_pool_size = 6


def train_single_param(lang, vocab_size):
    """ Train a single parameter combination"""
    
    start = time.time()
    model_prefix = "cc100_"+lang + "_vocab_" + str(vocab_size)
    spm.SentencePieceTrainer.train(input=sentences_fp[lang]
                                       , model_prefix=model_prefix
                                       , vocab_size=vocab_size
                                       , character_coverage = character_coverages[lang]
                                      ) 
    print("Trained {} in {} seconds".format(model_prefix, time.time()-start))
    


#loop over parameters for the actual train in multiprocessing pool
#param_combinations = [(lang,vocab_size) for lang in languages for vocab_size in vocab_sizes]
#p = Pool(multiproc_pool_size)

#p.starmap(train_single_param, param_combinations)